In [ ]:
import os
import sys
import shutil

If you use yaml file to set parameters, use below cell as Code.

import yaml
conf_file = "conf.yaml"
with open(conf_file, "r") as f:
    conf = yaml.safe_load(f)

If you don't use yaml file to set parameters, use below.


|  key  |  description  |
| :---: | :--- |
|use_devices | use gpu number. If you want to do distributed learning, write like "0,1,2" |
|model_dir                                           |all outputs are written here. <br>e.g., trained model, inference results, etc..|
|  train_x_paths<br> valid_x_paths<br> test_x_paths  |  directory where input images exist.|
|  train_y_paths<br> valid_y_paths                   |  directory where segmentation image exsit.<br> Each segmentation image name must be the same for corresponding input image.  |
|which_to_inference <br> which_to_visualise          | chosse from "train", "valid", "test".|
|output_activation                                   | chosse "softmax" or "sigmoid". <br>softmax means each pixcel is assigned to 1 label.<br>sigmoid means each pixel can assigned 1 or more labels.|
|batch_size | batch size|
|n_epochs   |nuber of epochs |
|image_size | [height, width] |
|optimizer  |"Adam" or "Nadam" or "SGD" |
|loss       |choose one .<br>"CE": cross entropy <br> "FL": focal loss <br>"GDL": generalized dice loss
|use_tensorboard| if you want to use, True. If not, False.|

In [ ]:
conf = dict()
conf["use_devices"] = "0"

conf["model_dir"] = "path/to/output_dir"
conf["label_file_path"] = "path/to/label_list.csv"

conf["train_x_paths"] = ["path/to/train_x1", "path/to/train_x2"]
conf["train_y_paths"] = ["path/to/train_y1", "path/to/train_y2"]

conf["valid_x_paths"] = ["path/to/valid_x1", "path/to/valid_x2"]
conf["valid_y_paths"] = ["path/to/valid_y1", "path/to/valid_y2"]

# below 3 lines are not needed in training phase.
# conf["test_x_paths"] = ["path/to/test_x1", "path/to/test_x2"] # if you don't have test data, enter empty list.([])
# conf["which_to_inference"] = ["train", "valid", "test"]
# conf["which_to_visualise"] = ["valid", "test"]

conf["output_activation"] = "softmax"

conf["batch_size"] = 8
conf["n_epochs"] = 1000
conf["image_size"] = [512, 512]
conf["optimzer"] = "Adam"
conf["loss"] = "FL"
conf["use_tensorboard"] = False


In [ ]:
use_devices = str(conf["use_devices"])
out_dir = conf["model_dir"]
label_file_path = conf["label_file_path"]

batch_size = conf["batch_size"]
n_epochs = conf["n_epochs"]
output_activation = conf["output_activation"]
image_size = conf["image_size"]
loss = conf["loss"]
optimizer = conf["optimizer"]
class_weight = conf["class_weight"]
use_tensorboard = conf["use_tensorboard"]

gpu setting

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = use_devices
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"

import modules

In [ ]:

import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras as keras

from deeplab_v3plus_tfkeras.model import deeplab_v3plus_transfer_os16
from deeplab_v3plus_tfkeras.metrics import make_IoU
from deeplab_v3plus_tfkeras.label import Label
from deeplab_v3plus_tfkeras.input_data_processing import make_xy_path_list
import deeplab_v3plus_tfkeras.data_gen as my_generator
import deeplab_v3plus_tfkeras.loss as my_loss_func
tf.compat.v1.enable_eager_execution()
matplotlib.use('Agg')

some settings

In [ ]:
label = Label(label_file_path)
if class_weight is not None:
    label.add_class_weight(class_weight)

n_gpus = len(use_devices.split(','))

batch_size = batch_size * n_gpus

os.makedirs(out_dir, exist_ok=True)

preprocess = keras.applications.xception.preprocess_input

make training dataset

In [ ]:
train_x_paths, train_y_paths = make_xy_path_list(conf["train_x_paths"], conf["train_y_paths"])
n_train_data = len(train_x_paths)
train_dataset, train_map_f = my_generator.make_path_generator(
    train_x_paths,
    train_y_paths,
    image_size,
    label,
    preprocess,
    augmentation=True,
    # augmentation=False,
    resize_or_crop="crop",
    data_type="image")

train_dataset = train_dataset.shuffle(n_train_data)
train_dataset = train_dataset.map(train_map_f, num_parallel_calls=tf.data.experimental.AUTOTUNE)
train_dataset = train_dataset.batch(batch_size)
train_dataset = train_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

make validation dataset

In [ ]:
valid_x_paths, valid_y_paths = make_xy_path_list(conf["valid_x_paths"], conf["valid_y_paths"])
n_valid_data = len(valid_x_paths)
valid_dataset, valid_map_f = my_generator.make_path_generator(
    valid_x_paths,
    valid_y_paths,
    image_size,
    label,
    preprocess,
    augmentation=False,
    resize_or_crop="crop",
    data_type="image")

valid_dataset = valid_dataset.map(valid_map_f, num_parallel_calls=tf.data.experimental.AUTOTUNE)
valid_dataset = valid_dataset.batch(batch_size)
valid_dataset = valid_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

define loss function

In [ ]:
if output_activation == "softmax":
    if loss == "CE":
        loss_function = tf.keras.losses.categorical_crossentropy
    elif loss == "FL":
        fl_alpha_list = [0.25] * label.n_labels
        fl_gamma_list = [2.0] * label.n_labels
        loss_function = \
            my_loss_func.make_focal_loss(label.n_labels,
                                         fl_alpha_list,
                                         fl_gamma_list)
    elif loss == "GDL":
        loss_function = my_loss_func.generalized_dice_loss
    else:
        raise Exception(loss + " is not supported.")

elif output_activation == "sigmoid":
    if loss == "CE":
        loss_function = keras.losses.binary_crossentropy
    elif loss == "FL":
        fl_alpha_list = [0.25] * label.n_labels
        fl_gamma_list = [2.0] * label.n_labels
        loss_function = \
            my_loss_func.make_focal_loss(label.n_labels,
                                         fl_alpha_list,
                                         fl_gamma_list)

    elif loss == "GDL":
        loss_function = my_loss_func.generalized_dice_loss
    else:
        raise Exception(loss + " is not supported.")


define optimizer

In [ ]:
if optimizer == "Adam":
    opt = tf.keras.optimizers.Adam()
elif optimizer == "Nadam":
    opt = tf.keras.optimizers.Nadam()
elif optimizer == "SGD":
    opt = tf.keras.optimizers.SGD()
else:
    raise Exception(
        "optimizer " + optimizer + " is not supported")

define metrics

In [ ]:
IoU = make_IoU(threshold=0.5)

make model

In [ ]:
layer_name_to_decoder = "block3_sepconv2_bn"
encoder_end_layer_name = "block13_sepconv2_bn"

if n_gpus >= 2:
    strategy = tf.distribute.MirroredStrategy()
    with strategy.scope():
        encoder = keras.applications.Xception(
            input_shape=(*image_size, 3),
            weights="imagenet",
            include_top=False)

        model = deeplab_v3plus_transfer_os16(
            label.n_labels,
            encoder,
            layer_name_to_decoder,
            encoder_end_layer_name,
            freeze_encoder=False,
            output_activation=output_activation,
            batch_renorm=False)

        model.compile(optimizer=opt,
                      loss=loss_function,
                      metrics=[IoU],
                      run_eagerly=True,
                      )
else:
    encoder = keras.applications.Xception(
        input_shape=(*image_size, 3),
        weights="imagenet",
        include_top=False)

    model = deeplab_v3plus_transfer_os16(
        label.n_labels,
        encoder,
        layer_name_to_decoder,
        encoder_end_layer_name,
        freeze_encoder=False,
        output_activation=output_activation,
        batch_renorm=False,
    )

    model.compile(optimizer=opt,
                  loss=loss_function,
                  metrics=[IoU],
                  run_eagerly=True,
                  )
model.summary()

make callbacks

In [ ]:
filepath = os.path.join(out_dir, 'best_model.h5')
cp_cb = keras.callbacks.ModelCheckpoint(
    filepath,
    monitor='val_IoU',
    verbose=1,
    save_best_only=True,
    save_weights_only=False,
    mode='max')
cbs = [cp_cb]

if use_tensorboard:
    log_dir = os.path.join(out_dir, "logs")
    TB_cb = tf.keras.callbacks.TensorBoard(
        log_dir=log_dir, write_graph=False)
    cbs.append(TB_cb)

    if os.path.exists(log_dir):
        shutil.rmtree(log_dir)

training. It fails sometimes. it may be due to a bug in tf2.2.0. After a few runs, it works.

In [ ]:
n_train_batch = int(np.ceil(n_train_data / batch_size))
n_valid_batch = int(np.ceil(n_valid_data / batch_size))
print("train batch:{}".format(n_train_batch))
print("valid batch:{}".format(n_valid_batch))
hist = model.fit(
    train_dataset,
    epochs=n_epochs,
    validation_data=valid_dataset,
    callbacks=cbs,
)

write training log

In [ ]:
hists = [hist.history["loss"],
         hist.history["val_loss"],
         hist.history["IoU"],
         hist.history["val_IoU"]]
hists = np.array(hists).T
hists_df = pd.DataFrame(hists, columns=["loss", "val_loss", "IoU", "val_IoU"])
hists_df.to_csv(os.path.join(out_dir, "training_log.csv"), index=False)

plt.figure(figsize=(20, 10))

plt.subplot(1, 2, 1)
plt.plot(hists_df["loss"], label="loss")
plt.plot(hists_df["val_loss"], label="val_loss")
plt.yscale("log")
plt.legend()
plt.grid()

plt.subplot(1, 2, 2)
plt.plot(hists_df["IoU"], label="IoU")
plt.plot(hists_df["val_IoU"], label="val_IoU")
plt.legend()
plt.grid()
plt.savefig(os.path.join(out_dir, 'losscurve.png'))

model.save(os.path.join(out_dir, 'final_epoch.h5'))
